In [ ]:
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt
agg = pd.read_parquet('../results/agg/summary_metrics.parquet')
# 1) Throughput scaling
sns.lineplot(data=agg, x='gpu_count', y='tokens_sec', hue='prompt_profile')
plt.title('Tokens/sec Scaling')
plt.savefig('../results/figs/throughput_scaling.png')
# 2) Latency CDF: raw per-request latency parquet 필요